In [ ]:
from math import nan
import pandas as pd
from charset_normalizer import from_path 
import re 
from pathlib import Path
import config
from config import *
from config import log_config
from eda_analisys import *
from openpyxl import load_workbook 

Config load

In [ ]:
# config
logger = log_config()
config.load_config() 

tables

In [ ]:
logger.info("Início do processamento")

df_eda = pd.read_excel(config.eda_sheet_full_path,sheet_name='tables')
df_eda.columns = df_eda.columns.str.strip().str.lower()

# Abre a planilha para saída
try:
    wb = load_workbook(config.eda_sheet_full_path)
    tables_sheet = wb["tables"]
    fields_sheet = wb["fields"]
except FileNotFoundError:
    print("Erro: Arquivo 'eda.xlsx' não encontrado.")
    exit(1)

# Monta df_fields
data = list(fields_sheet.values)  # retorna tuplas, incluindo cabeçalho
headers = data[0]
rows = data[1:]
df_fields = pd.DataFrame(rows, columns=headers)

header_tables = headers = {str(cell.value).lower(): idx+1 for idx, cell in enumerate(tables_sheet[1]) if cell.value}

for row in tables_sheet.iter_rows(min_row=2):

    table_name = row[header_tables['table']-1].value
    
    if not pd.isna(table_name): 

        print(table_name)

        data_file_full_path = Path(config.data_file_path / row[header_tables['file']-1].value)     

        # file size
        row[header_tables['file_size']-1].value = file_size(table_name, data_file_full_path)

        # lines (qty) 
        row[header_tables['lines']-1].value = lines(table_name,data_file_full_path)

        # columns (qty) 
        row[header_tables['cols']-1].value = columns(table_name,data_file_full_path,config.separator)

        # columns existance
        row[header_tables['col_exists']-1].value = column_exists(table_name,df_fields,data_file_full_path,config.separator)

        # columns existance
        row[header_tables['col_unique']-1].value = column_unique(table_name, data_file_full_path,config.separator)        

        # pk unique
        row[header_tables['pk_unique']-1].value = pk_unique(table_name,df_fields, data_file_full_path,config.separator)  

        # fields analisys
        filter_for_table = df_fields['table'] == table_name
        df_table_fields = df_fields[filter_for_table]
        for index, field_row in df_table_fields.iterrows(): 
            
            field_name = field_row['field']
            field_type = field_row['type']
            regex_value = field_row['regex']
            values_list = field_row['list']
            ranges_list = field_row['range'] 
            
            excel_row = index + 2

            # field_values (low, high, empty, nulls, zero, negative)
            excel_col_low = df_fields.columns.get_loc('lowest') + 1 
            excel_col_hig = df_fields.columns.get_loc('highest') + 1 
            excel_col_empty = df_fields.columns.get_loc('emptys') + 1 
            excel_col_nulls = df_fields.columns.get_loc('nulls') + 1 
            excel_col_zero = df_fields.columns.get_loc('zeroes') + 1 
            excel_col_negative = df_fields.columns.get_loc('negatives') + 1 
            excel_col_uniques = df_fields.columns.get_loc('uniques') + 1 
            excel_col_valid_format = df_fields.columns.get_loc('valid_format') + 1 
            excel_col_valid_list = df_fields.columns.get_loc('valid_list') + 1 
            excel_col_valid_range = df_fields.columns.get_loc('valid_range') + 1             

            low, high, emptys, nulls, zeroes, negatives, uniques, valid_formats, valid_list, valid_range  = field_values(table_name, field_name, regex_value, values_list, ranges_list, data_file_full_path, config.separator) 
            fields_sheet.cell(row=excel_row, column=excel_col_low).value = low 
            fields_sheet.cell(row=excel_row, column=excel_col_hig).value = high       
            fields_sheet.cell(row=excel_row, column=excel_col_empty).value = emptys
            fields_sheet.cell(row=excel_row, column=excel_col_nulls).value = nulls 
            fields_sheet.cell(row=excel_row, column=excel_col_zero).value = zeroes 
            fields_sheet.cell(row=excel_row, column=excel_col_negative).value = negatives 
            fields_sheet.cell(row=excel_row, column=excel_col_uniques).value = uniques 
            fields_sheet.cell(row=excel_row, column=excel_col_valid_format).value = valid_formats
            fields_sheet.cell(row=excel_row, column=excel_col_valid_list).value = valid_list
            fields_sheet.cell(row=excel_row, column=excel_col_valid_range).value = valid_range            

    wb.save(config.eda_sheet_full_path)
